# Subclassing `ASFProduct`

`ASFProduct` is the base class for all search result objects as of asf-search v7.0.0. There are several subclasses of `ASFProduct` that asf-search uses for specific platforms and product types with unique properties/functionality.

In [ ]:
import asf_search as asf
products = ['S1A_IW_SLC__1SDV_20231224T032123_20231224T032150_051791_064179_16F5-SLC', 'S1_185682_IW2_20210224T161634_VV_035E-BURST','S1-GUNW-D-R-087-tops-20190301_20190223-161540-20645N_18637N-PP-7a85-v2_0_1-unwrappedPhase','ALPSRP111041130-RTC_HI_RES']
results = asf.product_search(product_list=products)
results

Notice the different type in the `results` list: `S1Product`, `S1BURSTProduct`, `ARIAS1GUNWProduct`, and `ALOSProduct`.
Each of these are subclasses of type `ASFProduct`.

Let's compare the `properties` of `S1Product` and `ALOSProduct`

In [ ]:
s1, s1Burst, ariaGunw, alos = results

def compare_properties(lhs: asf.ASFProduct, rhs: asf.ASFProduct):
    # Compares properties of two ASFProduct objects in a color coded table
    # values printed in red are missing from that product type altogether
    
    # Color Coding
    RED = '\033[31m'
    GREEN = '\033[32m'
    BLUE  = '\033[34m'
    RESET = '\033[0m'

    print(f'\t{GREEN}{type(lhs)}{RESET}\t{BLUE}{type(rhs)}{RESET}')
    
    keys = {*lhs.properties.keys(), *rhs.properties.keys()}
    for key in keys:
        print(f"{key}:\n\t{GREEN}{lhs.properties.get(key, f'{RED}None')}{RESET}\t{BLUE}{rhs.properties.get(key, f'{RED}None')}{RESET}\n")

compare_properties(s1, alos)

Notice a few properties (marked in red) are missing from each product properties dict. For example, `S1Product` has `pgeVersion`, while `ALOSProduct` has `offNadirAngle`, `faradayRotation`, and `insarStackId`. Moreover, their `baseline` field differs.

In [ ]:
print(f"{s1.properties['fileID']}\n\t{s1.baseline}\n")
print(f"{alos.properties['fileID']}\n\t{alos.baseline}")

`ASFProduct` has a class enum, `BaselineCalcType` that determines how asf-search will handle perpendicular stack calculations. Each subclass keeps track of their baseline calculation type via `ASFProduct.baseline_type`

The three `BaselineCalcType` types:
- `NONE` Cannot be used in baseline calculations
- `PRE_CALCULATED` Has pre-calculated insarBaseline value that will be used for perpendicular calculations
- `CALCULATED` Uses position/velocity state vectors and ascending node time for perpendicular calculations

Any subclass object that changes `baseline_type` from the default of `BaselineCalcType.NONE` is elligble for building a baseline stacking with `ASFProduct.stack()` (see the 4-Baseline_Search.ipynb example notebook for more examples of baseline stacking).

In [ ]:
print(f"Baseline Calculation Types")
print(f"ASFProduct:\t {asf.ASFProduct.baseline_type}")
print(f"ALOSProduct:\t {alos.baseline_type}")
print(f"S1Product:\t {s1.baseline_type}")

`ASFProduct` subclasses even have their own stack search option methods. The `ASFProduct` implementation of `get_stack_opts()` returns `None`, but subclasses like `S1Product` and `ALOSProduct` have different approaches.

In [ ]:
print(f"S1Product:\n{s1.get_stack_opts()}\n")
print(f"ALOSProduct:\n{alos.get_stack_opts()}")

Sublcasses of type `ASFProduct` can just as easily be parent classes to other subclasses, like `S1Product`, which is the parent class to `S1BurstProduct` and `ARIAS1GUNWProduct`.

In [ ]:
print("S1BurstProduct:")
print(f"\tburst dict:\n\t{s1Burst.properties['burst']}")
print(f"\nS1BurstProduct.get_stack_opts(): {s1Burst.get_stack_opts()}\n\n")

print(f"ARIAS1GUNWProduct:")
print(f"\tperpendicularBaseline: {ariaGunw.properties['perpendicularBaseline']}")
print(f"\tOrbit: {ariaGunw.properties['orbit']}")

Because `ASFProduct` is built for subclassing, that means users can provide their own custom subclasses.

In [ ]:
import copy
from typing import Type, Union
from asf_search import ASFSearchOptions, ASFSession
from asf_search.ASFSearchOptions import ASFSearchOptions
from asf_search.CMR.translate import try_parse_int
from datetime import datetime

class MyCustomS1Subclass(asf.S1Product):
    def __init__(
                #default ASFProduct constructor arguments
                self, args: dict = {}, session: ASFSession = ASFSession()
            ):
        super().__init__(args, session)

        # totaly unique property of MyCustomClass
        self.timestamp = datetime.now()

    # _base_properties is a special dict of ASFProduct that maps keywords to granule UMM json
    # defining properties and their paths here in conjunction with `get_property_paths()` 
    # will let you easily access them in the product's `properties` dictionary
    # see `ASFProduct.umm_get()` for explanation of pathing
    _base_properties = {
        # Most product types use `CENTER_ESA_FRAME` as the value for `frameNumber` (unlike S1 and ALOS, which use `FRAME_NUMBER`), 
        # this creates a new `esaFrame` property so we have that value too
        'esaFrame': {'path': ['AdditionalAttributes', ('Name', 'CENTER_ESA_FRAME'), 'Values', 0], 'cast': try_parse_int}, #Sentinel and ALOS product alt for frameNumber (ESA_FRAME)
    }

    """ Example umm that the above pathing would map to:
        'umm': {
            'AdditionalAttributes': [
                {
                    'Name': 'CENTER_ESA_FRAME',
                    "Values": ['1300'] 
                },
                ...
            ],
            ...
        }
    """

    # write custom methods
    def as_umm_json(self) -> dict:
        return { 'umm': self.umm, 'meta': self.meta }
    
    # Or Override built in ASFProduct methods, like `geojson()`, `get_stack_opts()`, or `get_default_baseline_product_type()`
    
    # This override of `geojson()` includes the product's state vectors in the final geojson output, 
    # along with a custom class field timestamp and what version of asf-search was used at runtime
    def geojson(self) -> dict:
        output = super().geojson()

        output['properties']['stateVectors'] = self.get_state_vectors()
        output['properties']['timestamp'] = str(self.timestamp)
        output['properties']['ASFSearchVersion'] = asf.__version__
        return output
    
    # This method is used internally by `ASFProduct.translate_product()` 
    # to traverse the granule UMM for each property's corresponding values
    @staticmethod
    def get_property_paths() -> dict:
        return {
            **asf.S1Product.get_property_paths(),
            **MyCustomS1Subclass._base_properties
        }
    
    # This method normally stacks the current product
    # in this version we search for every SLC-BURST product that
    # overlaps the given area, and return a list of burst stacks
    def stack(self, 
            opts: ASFSearchOptions = None,
            aoi: str = None,
            useSubclass: Type[asf.ASFProduct] = None):
        
        bursts = asf.search(
            groupID=self.properties['groupID'], 
            processingLevel=asf.PRODUCT_TYPE.BURST,
            intersectsWith=aoi if aoi is not None else opts.intersectsWith
        )

        if len(bursts) == 0: # use default S1Product version if there's no SLC_BURST
            return super().stack(opts=opts, useSubclass=useSubclass)
        
        return [burst.stack(opts=opts) for burst in bursts]

customS1SubclassProduct = MyCustomS1Subclass({'umm': s1.umm, 'meta': s1.meta}, session=s1.session)

customS1SubclassProduct.geojson()

Notice the `customProperties` field in the output from `geojson()`.

In [ ]:
fairbanks_area = 'LINESTRING(-147.2885 64.7464,-147.733 64.8586,-148.1878 64.731)'
customStack = customS1SubclassProduct.stack(aoi=fairbanks_area)
customStack

In [ ]:
from typing import List

def view_stack_of_stacks(stack_of_stacks: List):
    print(f'Found {len(stack_of_stacks)} SLC-BURST stacks over AOI, stack lengths:')
    for stack_idx, stack in enumerate(stack_of_stacks):
        print(f"\t{stack_idx+1}:\t{len(stack)} SLC-BURSTs \t(Full Burst ID: {stack[-1].properties['burst']['fullBurstID']})")

view_stack_of_stacks(customStack)

In [ ]:
compare_properties(s1, customS1SubclassProduct)

# Using Custom `ASFProduct` Subclasses in Baseline Search

There may be instances where you want to build a spatio-temporal baseline stack from a reference of a custom subclass. `stack_from_id()` and `ASFProduct.stack()` support this via the `ASFProductSubclass` keyword.

In [ ]:
opts = asf.ASFSearchOptions(intersectsWith=fairbanks_area) # our custom class will be able to use our aoi this way

customSubclassStack = asf.stack_from_id('S1A_IW_SLC__1SDV_20231224T032123_20231224T032150_051791_064179_16F5-SLC', opts=opts, useSubclass=MyCustomS1Subclass)

view_stack_of_stacks(customSubclassStack)